In [1]:
# ==========================================
# File: 01_feature_engineering.ipynb (Partner Edition)
# ==========================================

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

# 1. 修正路径
if 'notebooks' in os.getcwd(): os.chdir('..')

# ========================================================
# 辅助函数：日志系统
# ========================================================
feature_log = {}
deleted_features = {}

def log_step(step_name, current_cols, dropped_cols=None):
    feature_log[step_name] = len(current_cols)
    if dropped_cols is not None and len(dropped_cols) > 0:
        deleted_features[step_name] = list(dropped_cols)
    print(f"   -> [{step_name}] Remaining: {len(current_cols)} (Dropped: {len(dropped_cols) if dropped_cols is not None else 0})")

# ========================================================
# 阶段 1: 生成原始特征
# ========================================================
print(">>> [Step 1] Reading Data & Generating Features...")

try:
    df = pd.read_csv('data/raw/cdft_qsar.CSV')
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到 data/raw/cdft_qsar.CSV")

target_col = 'log(1o2)'
cols_to_drop = [target_col, 'ID', 'dup_count']
X_raw_cdft = df.drop(columns=[c for c in cols_to_drop if c in df.columns]).select_dtypes(include=[np.number])
y_all = df[target_col]
smiles_all = df['SMILES']

# RDKit 生成
desc_names = [x[0] for x in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
mols = [Chem.MolFromSmiles(s) for s in smiles_all]
valid_idx = [i for i, m in enumerate(mols) if m is not None]
X_raw_rdkit = pd.DataFrame([calc.CalcDescriptors(mols[i]) for i in valid_idx], columns=desc_names)

# 合并与清洗 Inf
X_full = pd.concat([X_raw_cdft.iloc[valid_idx].reset_index(drop=True),
                    X_raw_rdkit.reset_index(drop=True)], axis=1)
y_full = y_all.iloc[valid_idx].reset_index(drop=True)
smiles_full = smiles_all.iloc[valid_idx].reset_index(drop=True)
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"📊 Raw Features Shape: {X_full.shape}")
log_step("Raw_Generated", X_full.columns)

# ========================================================
# 阶段 2: 动态切分 (Dynamic Splitting)
# ========================================================
print("\n>>> [Step 2] Dynamic Splitting (Train/Val/Test)...")

data_w_meta = X_full.copy()
data_w_meta['Target_Log1o2'] = y_full
data_w_meta['SMILES'] = smiles_full

# ★★★ 尝试读取外部种子 ★★★
seed_file = "current_seed_config.txt"
if os.path.exists(seed_file):
    with open(seed_file, "r") as f:
        try:
            CURRENT_SEED = int(f.read().strip())
            print(f"📨 收到外部指令: 使用随机种子 Seed = {CURRENT_SEED}")
        except:
            CURRENT_SEED = 888 # 读取失败时的兜底
            print(f"⚠️ 读取指令失败，使用默认种子 Seed = {CURRENT_SEED}")
else:
    CURRENT_SEED = 888 # 没纸条时的默认值
    print(f"⚠️ 未收到指令，使用默认种子 Seed = {CURRENT_SEED}")

# 使用读取到的 CURRENT_SEED 进行切分 (70/15/15)
train_val_set, test_set = train_test_split(data_w_meta, test_size=0.15, random_state=CURRENT_SEED)
train_set, val_set = train_test_split(train_val_set, test_size=15/85, random_state=CURRENT_SEED)

def split_features(df):
    y = df['Target_Log1o2']
    meta = df['SMILES']
    X = df.drop(columns=['Target_Log1o2', 'SMILES'])
    return X, y, meta

X_train, y_train, s_train = split_features(train_set)
X_val,   y_val,   s_val   = split_features(val_set)
X_test,  y_test,  s_test  = split_features(test_set)

# ========================================================
# 阶段 3: 严格防泄漏清洗流程 (Fit on Train ONLY)
# ========================================================
print("\n>>> [Step 3] Processing Pipeline (Fit on Train)...")

# 1. Imputer (Median)
print("   Processing Imputation (Median)...")
imputer = SimpleImputer(strategy='median')
imputer.fit(X_train)
cols = X_train.columns
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns=cols)
X_val_imp   = pd.DataFrame(imputer.transform(X_val),   columns=cols)
X_test_imp  = pd.DataFrame(imputer.transform(X_test),  columns=cols)

# 2. Quasi-Constant Filter
print("   Processing Quasi-Constant Filter (threshold=0.99)...")
def get_quasi_constant_cols(df, threshold=0.99):
    to_drop = []
    for col in df.columns:
        if df[col].value_counts(normalize=True).iloc[0] > threshold:
            to_drop.append(col)
    return to_drop

vars_to_drop = get_quasi_constant_cols(X_train_imp)
X_train_var = X_train_imp.drop(columns=vars_to_drop)
X_val_var   = X_val_imp.drop(columns=vars_to_drop)
X_test_var  = X_test_imp.drop(columns=vars_to_drop)
log_step("Variance_Filter", X_train_var.columns, dropped_cols=vars_to_drop)

# 3. Correlation Filter (你的建议1：改进版)
# 我们使用 np.number 包含整数（如环数），防止计数型特征共线性
print("   Processing Correlation Filter (threshold=0.90)...")
numeric_cols = X_train_var.select_dtypes(include=[np.number]).columns
corr_matrix = X_train_var[numeric_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
corr_to_drop = [col for col in upper.columns if any(upper[col] > 0.90)]

X_train_corr = X_train_var.drop(columns=corr_to_drop)
X_val_corr   = X_val_var.drop(columns=corr_to_drop)
X_test_corr  = X_test_var.drop(columns=corr_to_drop)
log_step("Correlation_Filter", X_train_corr.columns, dropped_cols=corr_to_drop)

# 4. ElasticNet Selection
print("   Processing ElasticNet Selection...")
scaler_temp = StandardScaler()
X_train_scaled = scaler_temp.fit_transform(X_train_corr)

l1_ratios = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0]
elastic = ElasticNetCV(l1_ratio=l1_ratios, cv=5, random_state=42, n_jobs=-1, max_iter=20000)
elastic.fit(X_train_scaled, y_train)

model_sel = SelectFromModel(elastic, prefit=True)
selected_mask = model_sel.get_support()
selected_feats = X_train_corr.columns[selected_mask]
dropped_elastic = X_train_corr.columns[~selected_mask]

# 你的建议2：最小特征保护
if len(selected_feats) < 5:
    raise RuntimeError(
        f"❌ CRITICAL ERROR: ElasticNet selected too few features ({len(selected_feats)}). "
        "Check data stability or adjust l1_ratio grid."
    )

X_train_final = X_train_corr[selected_feats]
X_val_final   = X_val_corr[selected_feats]
X_test_final  = X_test_corr[selected_feats]

log_step("ElasticNet_Selection", X_train_final.columns, dropped_cols=dropped_elastic)

# ========================================================
# 阶段 4: 保存与日志 (你的建议3 & 4)
# ========================================================
print("\n>>> [Step 4] Saving Results & Logs...")
os.makedirs('data/processed', exist_ok=True)
os.makedirs('results/logs', exist_ok=True)

# 保存数据
def save_clean_csv(X, y, s, filename):
    df_out = X.copy()
    df_out['Target_Log1o2'] = y.values
    df_out['SMILES'] = s.values
    df_out.to_csv(filename, index=False)

save_clean_csv(X_train_final, y_train, s_train, 'data/processed/train_data.csv')
save_clean_csv(X_val_final,   y_val,   s_val,   'data/processed/val_data.csv')
save_clean_csv(X_test_final,  y_test,  s_test,  'data/processed/test_data.csv')

# 你的建议3：保存特征数变化日志
pd.DataFrame(
    feature_log.items(),
    columns=["Step", "Remaining_Features"]
).to_csv("results/logs/feature_count_by_step.csv", index=False)

# 你的建议4：保存 ElasticNet 参数
with open("results/logs/elasticnet_params.txt", "w") as f:
    f.write(f"alpha: {elastic.alpha_}\n")
    f.write(f"l1_ratio: {elastic.l1_ratio_}\n")
    f.write(f"n_features_in: {len(X_train_corr.columns)}\n")
    f.write(f"n_features_out: {len(selected_feats)}\n")

# 保存特征清单
pd.DataFrame(selected_feats, columns=['Selected_Features']).to_csv('results/logs/final_feature_list.csv', index=False)

print(f"✅ Success! ElasticNet kept {len(selected_feats)} robust features.")
print(f"   Alpha: {elastic.alpha_:.6f}, L1_Ratio: {elastic.l1_ratio_}")
print("   Logs saved to results/logs/")

>>> [Step 1] Reading Data & Generating Features...


📊 Raw Features Shape: (151, 240)
   -> [Raw_Generated] Remaining: 240 (Dropped: 0)

>>> [Step 2] Dynamic Splitting (Train/Val/Test)...
📨 收到外部指令: 使用随机种子 Seed = 288

>>> [Step 3] Processing Pipeline (Fit on Train)...
   Processing Imputation (Median)...
   Processing Quasi-Constant Filter (threshold=0.99)...
   -> [Variance_Filter] Remaining: 188 (Dropped: 52)
   Processing Correlation Filter (threshold=0.90)...
   -> [Correlation_Filter] Remaining: 126 (Dropped: 62)
   Processing ElasticNet Selection...


   -> [ElasticNet_Selection] Remaining: 21 (Dropped: 105)

>>> [Step 4] Saving Results & Logs...
✅ Success! ElasticNet kept 21 robust features.
   Alpha: 0.231579, L1_Ratio: 0.5
   Logs saved to results/logs/
